In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
!pip install -r requirements.txt
!pip install ipywidgets -q
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 11.1 MB 711 kB/s eta 0:00:01
Processing ./.cache/pip/wheels/84/46/04/7fc092936deb2b62c6eb14c1b92d79059ca697be3d423232e1/bert_extractive_summarizer-0.5.0-py3-none-any.whl
  Using cached numpy-1.16.3-cp37-cp37m-manylinux1_x86_64.whl (17.3 MB)
  Using cached torch-1.0.1-cp37-cp37m-manylinux1_x86_64.whl (560.1 MB)
  Using cached spacy-2.1.3-cp37-cp37m-manylinux1_x86_64.whl (27.7 MB)
  Using cached transformers-2.2.2-py3-none-any.whl (387 kB)
  Using cached Cython-0.29.10-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Using cached tqdm-4.32.2-py2.py3-none-any.whl (50 kB)
  Using cached neuralcoref-4.0-cp37-cp37m-manylinux1_x86_64.whl (286 kB)
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Using cached plac-0.9.6-py2.py3-none-any.whl (20 kB)
  Using cached jsonschema-2.6.0-py2.py3-none-any.whl (39 kB)
  Using cached wasabi-0.8.0-py3-none-any.whl (23 kB)
  Using cached blis-0.2.4-cp37-cp37m-manylinux1_x86_64.whl (3.2 MB)
  Using cached murmur

In [4]:
from transformers import *
custom_config = BertConfig.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')
custom_config.output_hidden_states=True

tokenizer = BertTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')

model = BertModel.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', config=custom_config)

In [5]:
from summarizer import Summarizer

summed = Summarizer(custom_model= model, custom_tokenizer=tokenizer)

In [6]:
import os

# Generate list of processed .txt files to summarize
docs = os.listdir("/root/MaMome_summarize/Filtered")


# Iterate through processed .txt files and generate summaries and write to file
for doc in docs:
    
    with open('/root/MaMome_summarize/Filtered/'+ doc, "r") as in_f:
        body = in_f.read()
        
    temp = body.split("_PARAGRAPHS", maxsplit = 1)
    
    paralength = (int(temp[0])//3) 
    
    if paralength < 3:
        sumsents = 3
    elif paralength > 8:
        sumsents = 8
    else:
        sumsents = paralength
    
    summary = ''.join(summed(body, num_sentences = 4))

    #summary = ''.join(summed(temp[1], num_sentences = 4))

    path = "/root/Summary/" + doc[:-4] + "_SUMMARY_terse.txt"

    with open(path, "w") as out_f:
        out_f.write(summary)

IsADirectoryError: [Errno 21] Is a directory: '/root/MaMome_summarize/Filtered/.ipynb_checkpoints'

In [25]:
# uploading to AWS S3 Bucket
print(os.getcwd())
s3 = boto3.resource('s3')

bucket = sess.default_bucket()   

txtfiles = os.listdir("/root/Summary")

fpath = os.getcwd() 

for txtfile in txtfiles:
    fpath = 'root/Summary/' + txtfile 
    s3.meta.client.upload_file(fpath, bucket, txtfile)


/root


FileNotFoundError: [Errno 2] No such file or directory: 'root/Summary/acute DSS induced colitis promotes gut microbial dysbiosis in mice_FILTERED_SUMMARY.txt'

In [17]:
bucket

'sagemaker-us-west-2-360604624615'

In [20]:
os.getcwd()

'/root'